**Build Video Classification Model Using Deep Learning**




Import Necessary Libraries

In [ ]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from PIL import Image
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import cv2
import math
import os
from scipy import stats as s

Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


storing the name of training videos in dataframe

In [ ]:
# open the .txt file which have names of training videos
f = open("/content/drive/MyDrive/project2/trainlist.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos

train = train[:-1]
train.head()

,video_name
0,BodyWeightSquats/v_BodyWeightSquats_g08_c01.avi 1
1,BodyWeightSquats/v_BodyWeightSquats_g08_c02.avi 1
2,BodyWeightSquats/v_BodyWeightSquats_g08_c03.avi 1
3,BodyWeightSquats/v_BodyWeightSquats_g08_c04.avi 1
4,BodyWeightSquats/v_BodyWeightSquats_g09_c01.avi 1


storing the name of testing videos in dataframe

In [ ]:
# open the .txt file which have names of test videos
f = open("/content/drive/MyDrive/project2/testlist.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,BodyWeightSquats/v_BodyWeightSquats_g01_c01.avi
1,BodyWeightSquats/v_BodyWeightSquats_g01_c02.avi
2,BodyWeightSquats/v_BodyWeightSquats_g01_c03.avi
3,BodyWeightSquats/v_BodyWeightSquats_g01_c04.avi
4,BodyWeightSquats/v_BodyWeightSquats_g02_c01.avi


Creating tags for both training and testing videos

In [ ]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [ ]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('/content/drive/MyDrive/project2/Train/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='/content/drive/MyDrive/project2/train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 185/185 [01:22<00:00,  2.24it/s]


In [ ]:
# getting the names of all the images
images = glob("/content/drive/MyDrive/project2/train_1/*.jpg")
train_image = []
train_class = []

for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[-1])
    # creating the class of image
    train_class.append(images[i].split('/')[-1].split('_')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('/content/drive/MyDrive/project2/train_new.csv',header=True, index=False)

100%|██████████| 897/897 [00:00<00:00, 523850.00it/s]


Training the video classification model

In [ ]:
#read the .csv file contents first 5 row
train = pd.read_csv('/content/drive/MyDrive/project2/train_new.csv')
train.head()

,image,class
0,v_BodyWeightSquats_g08_c01.avi_frame0.jpg,BodyWeightSquats
1,v_BodyWeightSquats_g08_c01.avi_frame1.jpg,BodyWeightSquats
2,v_BodyWeightSquats_g08_c01.avi_frame2.jpg,BodyWeightSquats
3,v_BodyWeightSquats_g08_c01.avi_frame3.jpg,BodyWeightSquats
4,v_BodyWeightSquats_g08_c02.avi_frame0.jpg,BodyWeightSquats


storing earlier extracted frames as a NumPy array

In [ ]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('/content/drive/MyDrive/project2/train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 897/897 [00:05<00:00, 157.70it/s]


(897, 224, 224, 3)

creating validation set

In [ ]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

For this particular dataset, we will be using the VGG-16 pre-trained model. Let’s create a base model of the pre-trained model:

In [ ]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


extract features from this pre trained model

In [ ]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(717, 7, 7, 512)

In [ ]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(180, 7, 7, 512)

We will use a fully connected network now to fine-tune the model. This fully connected network takes input in single dimension. So, we will reshape the images into a single dimension

In [ ]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(717, 7*7*512)
X_test = X_test.reshape(180, 7*7*512)

normalize the pixel values, i.e., keep the pixel values between 0 and 1. This helps the model to converge faster.

In [ ]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [ ]:
# shape of images
X_train.shape

(717, 25088)

Creating architecture

In [ ]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

Build function to save the weights of the model

In [ ]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])


In [ ]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
6/6 [==============================] - 4s 562ms/step - loss: 0.7340 - accuracy: 0.5314 - val_loss: 0.5900 - val_accuracy: 0.7667
Epoch 2/200
6/6 [==============================] - 3s 516ms/step - loss: 0.6408 - accuracy: 0.6499 - val_loss: 0.3445 - val_accuracy: 0.8778
Epoch 3/200
6/6 [==============================] - 3s 459ms/step - loss: 0.3672 - accuracy: 0.8257 - val_loss: 0.0739 - val_accuracy: 0.9944
Epoch 4/200
6/6 [==============================] - 3s 439ms/step - loss: 0.1236 - accuracy: 0.9609 - val_loss: 0.0050 - val_accuracy: 1.0000
Epoch 5/200
6/6 [==============================] - 2s 306ms/step - loss: 0.0327 - accuracy: 0.9888 - val_loss: 0.0148 - val_accuracy: 0.9944
Epoch 6/200
6/6 [==============================] - 3s 462ms/step - loss: 0.0129 - accuracy: 0.9958 - val_loss: 4.9443e-05 - val_accuracy: 1.0000
Epoch 7/200
6/6 [==============================] - 2s 304ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 4.5631e-04 - val_accuracy: 1.0000
Epoch

Evaluating our Video Classification Model

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)

his is the pre-trained model and we will fine-tune it

In [ ]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'], run_eagerly=True)

Creating the test data

In [ ]:
# getting the test list
f = open("/content/drive/MyDrive/project2/testlist.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,BodyWeightSquats/v_BodyWeightSquats_g01_c01.avi
1,BodyWeightSquats/v_BodyWeightSquats_g01_c02.avi
2,BodyWeightSquats/v_BodyWeightSquats_g01_c03.avi
3,BodyWeightSquats/v_BodyWeightSquats_g01_c04.avi
4,BodyWeightSquats/v_BodyWeightSquats_g02_c01.avi


In [ ]:
# creating the tags
train = pd.read_csv('/content/drive/MyDrive/Project/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

Generating predictions for test videos

In [ ]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('/content/drive/MyDrive/project2/Test/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('/content/drive/MyDrive/Project/temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='/content/drive/MyDrive/Project/temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("/content/drive/MyDrive/Project/temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    #prediction = model.predict_classes(prediction_images) 
    prediction = (model.predict(prediction_images) > 0.5).astype("int32")
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])


100%|██████████| 66/66 [03:28<00:00,  3.16s/it]


Evaluating the model

In [ ]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100
